# Milestone 2 - Tool Comparison
## Name: Alexander James, Joshua Ludolf, & Matthew Trevino
``Date: 02-25-2025``
### Description of this file:
This Jupyter Notebook provides a comprehensive walkthrough of using the `NFStream`, `Pyshark` & `Scapy`  library for network traffic analysis. Additionally noting Strengths & Weaknesses. The notebook includes the following sections:

1. **Installation of Requirements**: Installing the necessary `NFStream`, `Pyshark` & `Scapy` library.
2. **Importing Libraries**: Importing the required libraries for network traffic analysis.
3. **Strengths & Weaknesses**: Detailed information about the strengths and weaknesses.

In [19]:
%pip install nfstream
%pip install scapy
%pip install pyshark
%pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.




## NFStream

### Strengths
- **Performance**: Highly optimized C++ core with Python bindings makes NFStream significantly faster for processing large pcap files or real-time traffic
- **Memory Efficiency**: Uses streaming approach that minimizes memory footprint compared to Scapy and PyShark
- **Flow-based Analysis**: Automatically handles flow creation and tracking, providing higher-level abstractions
- **Built-in Feature Extraction**: Provides 100+ pre-built traffic features out of the box
- **ML Integration**: Designed to work well with machine learning pipelines

### Weaknesses
- **Less Flexibility**: Offers fewer packet manipulation capabilities compared to Scapy
- **Learning Curve**: Flow-based paradigm may require adjustment for those used to packet-level analysis
- **Limited Protocol Support**: Supports fewer application protocols than PyShark (which leverages Wireshark dissectors)
- **Python Version**: Only operates in python 3.11 or 3.8 and below (other version failed in build on github)

In [20]:
import subprocess

result = subprocess.run(['ipconfig', '/all'], capture_output=True, text=True)
print(result.stdout)


Windows IP Configuration

   Host Name . . . . . . . . . . . . : DESKTOP-92U6QUF
   Primary Dns Suffix  . . . . . . . : 
   Node Type . . . . . . . . . . . . : Hybrid
   IP Routing Enabled. . . . . . . . : No
   WINS Proxy Enabled. . . . . . . . : No
   DNS Suffix Search List. . . . . . : wirelessinternet

Ethernet adapter Ethernet 2:

   Media State . . . . . . . . . . . : Media disconnected
   Connection-specific DNS Suffix  . : 
   Description . . . . . . . . . . . : ExpressVPN TAP Adapter
   Physical Address. . . . . . . . . : 00-FF-68-7E-5D-82
   DHCP Enabled. . . . . . . . . . . : Yes
   Autoconfiguration Enabled . . . . : Yes

Ethernet adapter Ethernet 3:

   Connection-specific DNS Suffix  . : 
   Description . . . . . . . . . . . : VirtualBox Host-Only Ethernet Adapter
   Physical Address. . . . . . . . . : 0A-00-27-00-00-14
   DHCP Enabled. . . . . . . . . . . : No
   Autoconfiguration Enabled . . . . : Yes
   Link-local IPv6 Address . . . . . : fe80::3641:f60f:7e24:ac5c%20(

In [21]:
from nfstream import NFStreamer
import time

# Set your Wi-Fi interface name. 
wifi_interface = "Intel(R) Wi-Fi 6 AX201 160MHz" 

# Create an NFStreamer object to capture live traffic with statistical analysis enabled.
# For live capture, set 'source' to the interface name.
streamer = NFStreamer(source=wifi_interface)

# Capture flows for a fixed duration (e.g., 0.5 second(s)) and then print each flow.
start_time = time.time()
flows = []

print(f"Capturing live traffic on interface {wifi_interface} for 0.5 second(s)...")
for flow in streamer:
    flows.append(flow)
    if time.time() - start_time > 0.5:
        break
for n in flows:
    print(f"Flow: {n}") 

Capturing live traffic on interface Intel(R) Wi-Fi 6 AX201 160MHz for 0.5 second(s)...


Flow: NFlow(id=0,
      expiration_id=0,
      src_ip=192.168.1.199,
      src_mac=54:14:f3:bb:2c:8b,
      src_oui=54:14:f3,
      src_port=51654,
      dst_ip=13.107.136.10,
      dst_mac=f8:ca:59:07:25:d5,
      dst_oui=f8:ca:59,
      dst_port=443,
      protocol=6,
      ip_version=4,
      vlan_id=0,
      tunnel_id=0,
      bidirectional_first_seen_ms=1742412861655,
      bidirectional_last_seen_ms=1742412931119,
      bidirectional_duration_ms=69464,
      bidirectional_packets=94,
      bidirectional_bytes=87361,
      src2dst_first_seen_ms=1742412861655,
      src2dst_last_seen_ms=1742412862087,
      src2dst_duration_ms=432,
      src2dst_packets=60,
      src2dst_bytes=82469,
      dst2src_first_seen_ms=1742412861698,
      dst2src_last_seen_ms=1742412931119,
      dst2src_duration_ms=69421,
      dst2src_packets=34,
      dst2src_bytes=4892,
      application_name=TLS,
      application_category_name=Web,
      application_is_guessed=0,
      application_confidence=6,
   

## Scapy

### Strengths
- **Performance**: Significantly faster than NFStream for large-scale analysis
- **Packet Creation/Manipulation**: Unmatched flexibility for crafting custom packets and protocols
- **Interactive Use**: Excellent for testing and experimenting with network protocols
- **Powerful Dissection**: Can decode a wide range of protocols with manual control
- **Scriptability**: Great for automating complex network tasks and penetration testing

### Weaknesses
- **Memory Usage**: Loads entire packet captures into memory
- **Steep Learning Curve**: Requires deep knowledge of protocol structures
- **Limited Scalability**: Not ideal for processing gigabytes of traffic data

In [22]:
from scapy.all import IP, TCP, send, sniff

# Create a custom packet
packet = IP(dst="192.168.1.1")/TCP(dport=80)/"GET / HTTP/1.1\r\nHost: 192.168.1.1\r\n\r\n"

# Send the packet
send(packet)

# Sniff and analyze packets
def packet_callback(packet):
    if packet.haslayer(TCP):
        print(f"Packet: {packet.summary()}")

sniff(iface="Intel(R) Wi-Fi 6 AX201 160MHz", prn=packet_callback, count=10)


Sent 1 packets.


<Sniffed: TCP:0 UDP:10 ICMP:0 Other:0>

## PyShark - See/execute shark.py for sample

### Strengths
- **Wireshark Integration**: Access to all Wireshark dissectors for comprehensive protocol support
- **Readable Output**: Human-friendly packet information similar to Wireshark GUI
- **Familiar Interface**: Easy transition for Wireshark users
- **Deep Packet Inspection**: Excellent for detailed protocol analysis

### Weaknesses
- **Performance**: Much slower than NFStream, relies on tshark processes
- **Resource Intensive**: High memory usage when dealing with large captures
- **Dependency on Wireshark**: Requires Wireshark/tshark installation
- **Limited Packet Creation**: Not designed for packet crafting like Scapy
- **Jupyter Notebook Limitations**: Often encounters execution issues in Jupyter Notebook 
        environments due to asynchronous processing requirements

In [ ]:
import pyshark
import threading

# Define the interface you want to capture from
interface = 'Wi-Fi'

# Create a live capture object
capture = pyshark.LiveCapture(interface=interface)

# Function to display packets
def display_packets(capture, packet_count=10):
    print("Starting live capture on interface:", interface)
    capture.sniff(packet_count=packet_count)
    for packet in capture._packets:
        print(packet)

# Run the function in a separate thread
capture_thread = threading.Thread(target=display_packets, args=(capture, 10))
capture_thread.start()

Starting live capture on interface: Wi-Fi


c:\Users\Joshu\OneDrive - Texas A&M University-San Antonio\Spring 2025\Computer Security\Milestone 2\.venv\Lib\site-packages\pyshark\packet\fields.py:87: RuntimeWarning: coroutine 'LiveCapture._get_tshark_process' was never awaited
  if hasattr(main_field, 'get_default_value'):
c:\Users\Joshu\OneDrive - Texas A&M University-San Antonio\Spring 2025\Computer Security\Milestone 2\.venv\Lib\site-packages\pyshark\packet\fields.py:87: RuntimeWarning: coroutine 'Capture.packets_from_tshark' was never awaited
  if hasattr(main_field, 'get_default_value'):


Packet (Length: 66)
Layer ETH
:	Destination: f8:ca:59:07:25:d5
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 54:14:f3:bb:2c:8b
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Type: IPv4 (0x0800)
	Stream index: 0
Layer IP
:	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 52
	Identification: 0xfb54 (64340)
	010. .... = Flags: 0x2, Don't fragment
	0... .... = Reserved bit: Not set
	.1.. .... = Don't fragment: Set
	..0. .... = More fragments: Not set
	...0 0000 0000 0000 = Fragment Offset: 0
	Time to Live: 128
	Protocol: TCP (6)
	Header Checksum